# Make bilby samples from a list of galaxy hosts

This is a script to make bilby samples from a list of galaxy hosts. 

- The list of galaxy hosts is provided in a csv
- Galaxies are ingested, and the following quantities are propagated to the bilby sample
    - ra
    - dec
    - dL prior needs to be updated for the SkySim cosmology

In [6]:
import bilby as bb
import numpy as np
import gwcosmo as gwc
import pandas as pd
import GCR
import GCRCatalogs as GCRCat
import matplotlib.pyplot as plt
import os
from astropy.time import Time

In [8]:
cat_name2 = "skysim5000_v1.2"
skysimCat = GCRCat.load_catalog(cat_name2) # Load the skysim catalog
hostDF = pd.read_csv("/global/homes/s/seanmacb/DESC/DESC-GW/gwStreetlights/data/mergers-w=Lum,n=1e6,FromSkySim50.csv") # Load the CBC catalog

In [14]:
prior = bb.gw.prior.BBHPriorDict(aligned_spin=False) # The bbh prior, spins misaligned
prior["luminosity_distance"] = bb.gw.prior.UniformSourceFrame(0,5000,cosmology=skysimCat.cosmology,name='luminosity_distance', latex_label='$d_L$', unit='Mpc', boundary=None) # Update the luminosity distance prior, based on 

11:51 bilby INFO    : No prior given, using default BBH priors in /pscratch/sd/s/seanmacb/gwCosmoDesc/lib/python3.10/site-packages/bilby/gw/prior_files/precessing_spins_bbh.prior.


In [16]:
prior.sample()

{'mass_ratio': 0.33514523190166257,
 'chirp_mass': 33.113416605777644,
 'luminosity_distance': 2531.566589069049,
 'dec': 1.4511228970982644,
 'ra': 6.148930147811117,
 'theta_jn': 0.7205784912041286,
 'psi': 1.9363274200091976,
 'phase': 3.3056026221779615,
 'a_1': 0.8757204514999432,
 'a_2': 0.5472787586809683,
 'tilt_1': 0.47286406125625646,
 'tilt_2': 1.8685624619003376,
 'phi_12': 5.563054904275639,
 'phi_jl': 5.2167111529765835}

In [21]:
skysimCat.cosmology.luminosity_distance(hostDF.loc[0]["redshiftHubble"])

<Quantity 23772.32428394 Mpc>

## Let's do the sampling this way
- For each entry in the CBC catalog
    - take a sample of the bilby prior, most parameters will be added
    - For ra/dec, use mra and mdec from the cbc catalog
    - For dL, just compute directly based on the cosmology of SkySim
- Add to the dataframe
- Save the dataframe to disk

In [38]:
keys = list(prior.sample())
keys.pop(3)
keys.pop(3)
injDict = {}
for k in keys:
    injDict[k] = []

In [ ]:
cnt = 0
for ids,row in hostDF.iterrows():
    thisSample = prior.sample()
    for k in keys:
        injDict[k].append(thisSample[k])
    injDict["luminosity_distance"].append(float(str(skysimCat.cosmology.luminosity_distance(row["redshiftHubble"])).split(" ")[0]))
    if cnt % 10000 == 0:
        print("{}% finished".format(cnt//10000))
    cnt+=1

0% finished
1% finished
2% finished
3% finished
4% finished
5% finished
6% finished
7% finished


In [ ]:
for v in injDict.values():
    print(np.shape(v))

In [ ]:
for k in injDict.keys():
    hostDF[k] = injDict[k]

In [ ]:
hostDF

## Save this

In [ ]:
saveColumns = hostDF.columns.values[1:]

In [ ]:
dataDir = "/global/homes/s/seanmacb/DESC/DESC-GW/gwStreetlights/data"
pd.to_csv(os.path.join(dataDir,"mergers-w=Lum,n=1e6,FromSkySim50_withBilby.csv"),hostDF,columns=saveColumns,index=False)